# Query Wikidata for Belgian politicians

In [20]:
from datetime import datetime as dt

from SPARQLWrapper import SPARQLWrapper, JSON

## Fonctions

In [21]:
def get_rows():
    """Retrieve results from SPARQL"""
    endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
    sparql = SPARQLWrapper(endpoint)

    statement = """
    SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath WHERE {
        ?person wdt:P27 wd:Q31 .
        ?person wdt:P21 wd:Q6581097 .
        ?person wdt:P106 wd:Q82955 .
        ?person wdt:P569 ?dateBirth .
        ?person wdt:P39 wd:Q33126365 .
        OPTIONAL {?person wdt:P570 ?dateDeath .}
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
    }
    ORDER BY ?personLabel
    """

    sparql.setQuery(statement)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    rows = results['results']['bindings']
    print(f"\n{len(rows)} Belgian politicians who also were bourgmestre found\n")
    return rows

In [22]:
def show(rows, name_filter=None, n=10):
    """Display n politicians (default=10)"""
    date_format = "%Y-%m-%dT%H:%M:%SZ"
    if name_filter:
        rows = [row for row in rows if name_filter in row['personLabel']['value'].lower()]
    print(f"Displaying the first {n}:\n")
    for row in rows[:n]:
        try:
            birth_date = dt.strptime(row['dateBirth']['value'], date_format)
            birth_year = birth_date.year
        except ValueError:
            birth_year = "????"
        try:
            death_date = dt.strptime(row['dateDeath']['value'], date_format)
            death_year = death_date.year
        except ValueError: # unknown death date
            death_year = "????"
        except KeyError: # still alive
            death_year = ""
        print(f"{row['personLabel']['value']} ({birth_year}-{death_year})")

## Obtenir une liste des politiciens belges depuis Wiki Data 

In [23]:
rows = get_rows()


18 Belgian politicians who also were bourgmestre found



## Imprimer les 30 premiers politiciens

In [24]:
show(rows, n=10)

Displaying the first 10:

Adolphe Max (1869-1939)
Charles-Joseph, 4th Duke d'Ursel (1777-1860)
Felix Vanderstraeten (1823-1884)
François-Xavier de Donnea (1941-)
Freddy Thielemans (1944-)
Guillaume Van Volxem (1791-1868)
Hervé Brouhon (1924-1993)
Jan Grauls (1887-1960)
Jan-Baptist Verlooy (1746-1797)
Jean de Locquenghien (1518-1574)
